# 02 — Data Cleaning & Merging

- Aggregate weekly stats to seasonal totals
- Classify positions into analysis groups
- Join performance data to contracts via gsis_id
- Apply minimum snap thresholds
- Output: analysis-ready merged dataset

In [1]:
import sys
sys.path.insert(0, '..')

import pandas as pd
import numpy as np
from src.data_loader import (
    load_weekly_stats, load_rosters, load_contracts,
    load_snap_counts, load_players, load_pfr_stats, DATA_DIR
)
from src.cleaning import (
    aggregate_weekly_to_seasonal, compute_rate_stats,
    build_pfr_id_map, aggregate_snap_counts,
    prepare_contracts, merge_all, get_analysis_ready,
    classify_position, POSITION_GROUP_MAP
)

pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 200)

E:\Python\Python38-32\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
E:\Python\Python38-32\lib\site-packages\numpy\.libs\libopenblas.D6ALFJ4QQDWP6YNOQJNPYL27LRE6SILT.gfortran-win32.dll
E:\Python\Python38-32\lib\site-packages\numpy\.libs\libopenblas_v0.3.21-gcc_8_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


## Load Raw Data

In [2]:
weekly = load_weekly_stats()
contracts_raw = load_contracts()
snaps_raw = load_snap_counts()
players = load_players()
pfr_def = load_pfr_stats('def')

print(f"Weekly stats: {weekly.shape}")
print(f"Contracts: {contracts_raw.shape}")
print(f"Snap counts: {snaps_raw.shape}")
print(f"Players: {players.shape}")
print(f"PFR Defense: {pfr_def.shape}")

Loading cached weekly_stats from G:\ai\nfl\data\weekly_stats.parquet
Loading cached contracts from G:\ai\nfl\data\contracts.parquet
Loading cached snap_counts from G:\ai\nfl\data\snap_counts.parquet
Loading cached players from G:\ai\nfl\data\players.parquet
Loading cached pfr_def from G:\ai\nfl\data\pfr_def.parquet
Weekly stats: (54479, 53)
Contracts: (50164, 25)
Snap counts: (250336, 16)
Players: (24356, 39)
PFR Defense: (6512, 30)


## Step 1: Aggregate Weekly → Seasonal

In [3]:
seasonal = aggregate_weekly_to_seasonal(weekly)
seasonal = compute_rate_stats(seasonal)

print(f"Seasonal stats: {seasonal.shape}")
print(f"Player-seasons: {len(seasonal):,}")
print(f"Unique players: {seasonal['player_id'].nunique():,}")
print(f"\nPositions: {sorted(seasonal['position'].dropna().unique())}")
seasonal.head(3)

Seasonal stats: (6125, 46)
Player-seasons: 6,125
Unique players: 1,978

Positions: ['C', 'CB', 'DB', 'DE', 'DT', 'FB', 'FS', 'G', 'HB', 'ILB', 'K', 'MLB', 'NT', 'OLB', 'OT', 'P', 'QB', 'RB', 'S', 'SS', 'T', 'TE', 'WR']


,player_id,season,player_name,position,position_group,recent_team,games_played,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_fumbles,sack_fumbles_lost,passing_first_downs,passing_2pt_conversions,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_first_downs,receiving_2pt_conversions,special_teams_tds,fantasy_points,fantasy_points_ppr,passing_epa,rushing_epa,receiving_epa,completion_pct,yards_per_attempt,td_rate,int_rate,passer_rating,yards_per_carry,catch_rate,yards_per_reception
0,00-0007091,2015,Matt Hasselbeck,QB,QB,IND,8,156,256,1690.0,9,5.0,16.0,3,2,85.0,0,16,15.0,0,0.0,0.0,2.0,0,0,0,0.0,0,0.0,0.0,0.0,0,0.0,91.099998,91.099998,-0.113060,-5.870652,0.0,60.937500,6.601562,3.515625,1.953125,83.951823,0.937500,NaN,NaN
1,00-0010346,2015,Peyton Manning,QB,QB,DEN,13,249,423,2788.0,11,18.0,25.0,3,1,136.0,2,11,4.0,0,2.0,1.0,1.0,0,0,0,0.0,0,0.0,0.0,0.0,0,0.0,119.919998,119.919998,-54.886452,-8.214953,0.0,58.865248,6.591017,2.600473,4.255319,69.538022,0.363636,NaN,NaN
2,00-0018227,2015,Charles Woodson,FS,DB,LV,1,0,0,0.0,0,0.0,0.0,0,0,0.0,0,1,-3.0,0,0.0,0.0,0.0,0,0,0,0.0,0,0.0,0.0,0.0,0,0.0,-0.300000,-0.300000,0.000000,-0.917858,0.0,NaN,NaN,NaN,NaN,NaN,-3.000000,NaN,NaN


## Step 2: Position Classification

In [4]:
seasonal['pos_group'] = seasonal['position'].apply(classify_position)
print("Position group distribution:")
print(seasonal['pos_group'].value_counts().to_string())
print(f"\nUnmapped (UNK): {(seasonal['pos_group'] == 'UNK').sum()}")

Position group distribution:
pos_group
WR    2174
RB    1594
TE    1205
QB     769
DB     133
P      112
OL      74
LB      37
DL      16
K       11

Unmapped (UNK): 0


## Step 3: Prepare Contracts

In [5]:
contracts = prepare_contracts(contracts_raw)
print(f"Contract-seasons: {len(contracts):,}")
print(f"Unique players: {contracts['gsis_id'].nunique():,}")
print(f"\nContract type distribution:")
print(contracts['contract_type'].value_counts().to_string())
print(f"\nSalary (% cap) stats:")
print(contracts['apy_cap_pct'].describe())
contracts.head(3)

Contract-seasons: 32,382
Unique players: 8,600

Contract type distribution:
contract_type
rookie     27284
veteran     5098

Salary (% cap) stats:
count    32382.000000
mean         0.012521
std          0.022390
min          0.001000
25%          0.003000
50%          0.004000
75%          0.009000
max          0.245000
Name: apy_cap_pct, dtype: float64


,gsis_id,season,contract_player_name,contract_position,contract_team,apy_cap_pct,apy,value,guaranteed,years,year_signed,contract_type
8006,00-0034353,2024,Anthony Miller,WR,Chiefs,0.004,1.1250,1.1250,0.0,1.0,2024,rookie
32916,00-0036239,2024,Jonathan Garvin,ED,49ers,0.001,0.3024,0.3024,0.0,1.0,2024,rookie
32918,00-0033964,2024,Al-Quadin Muhammad,ED,Lions,0.001,0.3024,0.3024,0.0,1.0,2024,rookie


## Step 4: Build ID Crosswalk & Aggregate Snaps

In [6]:
pfr_to_gsis = build_pfr_id_map(players)
print(f"ID crosswalk: {len(pfr_to_gsis):,} pfr_id → gsis_id mappings")

seasonal_snaps = aggregate_snap_counts(snaps_raw, pfr_to_gsis)
print(f"Seasonal snap counts: {len(seasonal_snaps):,}")
print(f"With gsis_id: {seasonal_snaps['gsis_id'].notna().sum():,}")
print(f"\nSnap distribution:")
print(seasonal_snaps['total_snaps'].describe())

ID crosswalk: 22,194 pfr_id → gsis_id mappings
Seasonal snap counts: 21,067
With gsis_id: 21,033

Snap distribution:
count    21067.000000
mean       459.925096
std        362.168745
min          1.000000
25%        142.000000
50%        388.000000
75%        727.000000
max       1509.000000
Name: total_snaps, dtype: float64


## Step 5: Merge Everything

In [7]:
merged = merge_all(
    seasonal_stats=seasonal,
    contracts=contracts,
    snap_counts=seasonal_snaps,
    pfr_def=pfr_def,
    pfr_to_gsis=pfr_to_gsis,
    players=players,
)

print(f"Merged dataset: {merged.shape}")
print(f"\nJoin rates:")
print(f"  Has salary data:    {merged['has_salary'].sum():>6,} / {len(merged):,} ({merged['has_salary'].mean():.1%})")
print(f"  Meets snap min:     {merged['meets_snap_threshold'].sum():>6,} / {len(merged):,} ({merged['meets_snap_threshold'].mean():.1%})")
print(f"  Both (analysis):    {(merged['has_salary'] & merged['meets_snap_threshold']).sum():>6,}")
print(f"\nPosition group distribution (merged):")
print(merged['pos_group'].value_counts().to_string())

Merged dataset: (12391, 73)

Join rates:
  Has salary data:    12,001 / 12,391 (96.9%)
  Meets snap min:     10,915 / 12,391 (88.1%)
  Both (analysis):    10,597

Position group distribution (merged):
pos_group
DB     2203
WR     2180
RB     1599
UNK    1490
DL     1447
LB     1257
TE     1211
QB      769
P       118
OL      103
K        14


In [8]:
analysis = get_analysis_ready(merged)

print(f"Analysis-ready dataset: {analysis.shape}")
print(f"\nPosition group counts:")
print(analysis['pos_group'].value_counts().to_string())
print(f"\nSeason coverage:")
print(analysis.groupby('season').size().to_string())

Analysis-ready dataset: (9140, 73)

Position group counts:
pos_group
DB    1953
WR    1792
DL    1257
RB    1249
LB    1126
TE    1063
QB     497
P       99
OL      92
K       12

Season coverage:
season
2015     422
2016     444
2017     492
2018     977
2019     981
2020    1014
2021    1071
2022    1322
2023    1258
2024    1159


## Step 6: Spot-check Key Players

In [9]:
# Spot-check: Patrick Mahomes
mahomes = analysis[analysis['player_name'].str.contains('Mahomes', case=False, na=False)]
print("Patrick Mahomes:")
if len(mahomes) > 0:
    print(mahomes[['season', 'passing_yards', 'passing_tds', 'passer_rating', 'apy_cap_pct', 'total_snaps']].to_string(index=False))
else:
    print("  Not found in analysis set")

print()

# Spot-check: Aaron Donald
donald = analysis[analysis['player_name'].str.contains('Donald', case=False, na=False)]
print("Aaron Donald:")
if len(donald) > 0:
    print(donald[['season', 'pos_group', 'apy_cap_pct', 'total_snaps', 'def_sacks', 'def_tackles']].to_string(index=False))
else:
    print("  Not found in analysis set")

Patrick Mahomes:
 season  passing_yards  passing_tds  passer_rating  apy_cap_pct  total_snaps
   2018         5670.0         53.0     112.193252        0.025       1171.0
   2019         4932.0         36.0     106.473714        0.025       1088.0
   2020         5590.0         42.0     105.333924        0.227       1214.0
   2021         5896.0         48.0     101.634615        0.227       1374.0
   2022         5953.0         48.0     106.433824        0.227       1310.0
   2023         5234.0         33.0      94.096291        0.227       1340.0
   2024         4607.0         31.0      94.496737        0.227       1279.0

Aaron Donald:
 season pos_group  apy_cap_pct  total_snaps  def_sacks  def_tackles
   2015        OL        0.039       1099.0        NaN          NaN
   2016        OL        0.040       1186.0        NaN          NaN
   2015        RB        0.023        171.0        NaN          NaN
   2015        TE        0.007        575.0        NaN          NaN
   2016     

## Save Analysis-Ready Dataset

In [10]:
output_path = DATA_DIR / 'analysis_ready.parquet'
analysis.to_parquet(output_path, engine='fastparquet', index=False)
print(f"Saved analysis-ready data: {output_path}")
print(f"  {analysis.shape[0]:,} player-seasons x {analysis.shape[1]} columns")

Saved analysis-ready data: G:\ai\nfl\data\analysis_ready.parquet
  9,140 player-seasons x 73 columns
